<a href="https://colab.research.google.com/github/EffiSciencesResearch/ML4G-2.0/blob/master/workshops/agents/agents_hard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
# LLM Agents

This notebook is an introduction to the openai & anthropics API and to the design of LLM-agents.

In the first part, your goal will be to make a chatbot that negotiates the price of a specific good with you, then against an other model. This could be part of a persuation benchmark, where we evaluate how well a LLM can drive the price down against an other LLM.


> ## Learning outcomes
> - Knowing how to use LLM API
> - Finding your way in the documentation
> - Understand how to make LLMs take actions
> - Experiment with prompt engineering and control LLM outputs

During the workshop, you will need the documentation 
- For the OpenAI API: https://platform.openai.com/docs/
- For the Anthropics API: https://docs.anthropic.com/claude/docs/

In [ ]:
try:
    import google.colab
except ImportError:
    pass
else:  # In colab
    %pip install openai anthropic

In [ ]:
import os
import re
import json
import openai
import anthropic
import tomllib
from typing import Callable

openai_key = os.environ.get("OPENAI_API_KEY") or input("OpenAI API Key")
anthropic_key = os.environ.get("ANTHROPIC_API_KEY") or input("Anthropic API Key")

openai_client = openai.Client(api_key=openai_key)
anthropic_client = anthropic.Client(api_key=anthropic_key)

In [ ]:
MODELS = [
    # Small, cheap and fast
    "claude-3-haiku-20240307",
    # Medium
    "gpt-3.5-turbo",
    "claude-3-sonnet-20240229",
    # Big, slow expensive and good
    "gpt-4-turbo-preview",
    "claude-3-opus-20240229",
]

CLAUDE_SMALL, GPT3, CLAUDE_MEDIUM, GPT4, CLAUDE_BIG = MODELS
MODEL = MODELS[1]

## Understanding OpenAI's API

According to their quickstart guide, the following is an example of how to use the API. 
Try to understand what each parameter does, change them and see what happens.

<details>
<summary>Why is the messages parameter a list? What are each of its elements?</summary>

`message` is a list of each message in a conversation. They correspond to one chat, with messages from the assistant and the user, as you would see in the ChatGPT interface. Under the hood, the API concatenates them, and include markers tokens to diferenciate between the roles of `"user"`, `"assistant"` and `"system"`.
</details>


In [ ]:
completion = openai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair.",
        },
        {
            "role": "user",
            "content": "Compose a poem that explains the concept of recursion in programming.",
        },
    ],
    max_tokens=100,
)

print(completion.choices[0].message.content)

## Chat human-LLM

We'll start be creating one function to handle all the details of the APIs, so that we can forget about them later and focus more on the logic.

**Important note**: When you develop applications, evaluations or benchmarks with LLMs it is important always test with the smallest model first, as they are much faster and cheaper. This let you do more and faster iterations. However, when you start to tweak prompts, you need to tweak your prompts for one specific LLM, as they all react differently. The best prompt on GPT3 can be quite bad on GPT4 and vice versa.

<!-- Start by having the function work for openai's models, test it on the cells bellow, and you can later come back and implement it for anthropic. The anthropic part is especially interesting when we get to make the two of them chat. Who's the most persuasive? -->
We have the function already implemented for anthropic, and it should work once you fill `message_dicts` with the correct format. 

In [ ]:
def generate_answer(system: str, *messages: str, model: str = "gpt-3.5-turbo") -> str:
    """
    Generate the next message from the specified model.

    Args:
        system: the system prompt to use
        messages: the content of all the messages in the conversation, the first
            message is always with the "user" role, then it alternates between
            "assistant" and "user"
        model: the name of the model to use.
    """

    # Convert the list of string messages to a list of dictionaries, with the role alternating between "user" and "assistant"
    message_dicts = []
    ...

    if "gpt" in model:
        # Use openai API
        ...

    elif "claude" in model:
        # Use anthropic API
        response = anthropic_client.messages.create(
            system=system,
            messages=message_dicts,
            model=model,
            max_tokens=1000,
        )
        return response.content[0].text

    else:
        raise ValueError(f"Unkown model: {model!r}")

<details>
<summary>Show solution</summary>

```python
def generate_answer(system: str, *messages: str, model: str = "gpt-3.5-turbo") -> str:
    """
    Generate the next message from the specified model.

    Args:
        system: the system prompt to use
        messages: the content of all the messages in the conversation, the first
            message is always with the "user" role, then it alternates between
            "assistant" and "user"
        model: the name of the model to use.
    """

    # Convert the list of string messages to a list of dictionaries, with the role alternating between "user" and "assistant"
    message_dicts = []
    for i, message_content in enumerate(messages):
        if i % 2 == 0:
            message_dicts.append(dict(role="user", content=message_content))
        else:
            message_dicts.append(dict(role="assistant", content=message_content))

    if "gpt" in model:
        # Use openai API
        message_dicts.insert(0, dict(role="system", content=system))
        response = openai_client.chat.completions.create(
            messages=message_dicts,
            model=model,
            max_tokens=1000,
        )
        return response.choices[0].message.content

    elif "claude" in model:
        # Use anthropic API
        response = anthropic_client.messages.create(
            system=system,
            messages=message_dicts,
            model=model,
            max_tokens=1000,
        )
        return response.content[0].text

    else:
        raise ValueError(f"Unkown model: {model!r}")```

</details>

Bonus for later: make the API stream the answer, so that you can print it as it is generated. You can either print it directly in the function or transform the function in a generator that yields strings.

In [ ]:
generate_answer(
    "Answer the questions for the user, always in 2 sentences and from the perspective of the french president",
    "What are counterintuitive ways to make the most out of a summer school?",
)

Now we need a loop to keep the discussion going and add the new messages to the discussion. 
A few points to have in mind:

<details>
<summary>How do you know when to stop the loop? Can it continue forever?</summary>

You can stop when the LLM says something like "Offer accepted", but this is not enough. If they forget their instructions, your code is going to run forever. You need to add either a maximum number of messages, or have ask the user (=you) to confirm they want to continue regularly.
</details>
<details>
<summary>
The messages for the API need to start with a message from the "user". Who is the user here, and how do you generate the first message?
</summary>

The user is the the other AI, there is no human in this scenario. The first message from the buyer can be hardcoded to "Hello" for instance, and this message is put only in the list of message sent to the API when generating vendors responses, and not when generating buyers responses.
</details>

Note: You may need to add a time.sleep() in the loop to avoid rate limits. Bonus: catch rate limits errors and wait for the exact time.

In [ ]:
VENDOR_PROMPT = r"""
You sell tables. You inherited all the tables imaginable would like to sell one, but need to sell it for as much as you can.
The person in front of you seems interested in a new table.

You can make formal offers by ending your message with "Offer: XXX€"
If you want to accept an offer from the buyer, end your message with "Offer accepted!".

Important: your goal is to negociate to have the highest final price possible.
"""

BUYER_PROMPT = r"""
You are looking to buy a nice table, for as cheap as possible.

You can make formal offers by ending your message with "Offer: XXX€"
If you want to accept an offer from the vendor, end your message with "Offer accepted!".

Important: your goal is to negociate to pay the lowest final price possible.
"""

STOP = "Offer accepted!"


def chat_two_llms(
    vendor_system: str,
    buyer_system: str,
    vendor_model: str = MODEL,
    buyer_model: str = MODEL,
    stop: str = STOP,
    max_turns: int = 4,
):
    """Print a dialoge between the 2 LLMs."""
    ...


chat_two_llms(VENDOR_PROMPT, BUYER_PROMPT)

<details>
<summary>Show solution</summary>

```python
def chat_two_llms(
    vendor_system: str,
    buyer_system: str,
    vendor_model: str = MODEL,
    buyer_model: str = MODEL,
    stop: str = STOP,
    max_turns: int = 4,
):
    """Print a dialoge between the 2 LLMs."""
    messages = []

    # Be sure that the functions doesn't call the API endlessly
    for _ in range(max_turns):
        # 1. Generate the first message from the vendor (remember, the vendor needs to answer a message. Which one?)
        response = generate_answer(vendor_system, "Hello!", *messages, model=vendor_model)
        # 2. Print and save the message
        print(f"\n++++ Vendor:\n{response}")
        messages.append(response)
        # 3. Check if the conversation should stop (aggreement reached)
        if stop in response:
            break

        # Do the same for the buyer, except for the first message
        response = generate_answer(buyer_system, *messages, model=buyer_model)
        print(f"\n---- Buyer\n{response}")
        messages.append(response)

        if stop and stop in response:
            break


chat_two_llms(VENDOR_PROMPT, BUYER_PROMPT)```

</details>

At how much was the agreement? Does it change when you change models? Compare with the other people in the room. Are bigger models better at persuation? 

This is the simplest model of chat interaction between two LLMs. In practice, we don't often make them chat to each other, but interesting papers have created [a village of LLMs](https://arxiv.org/abs/2304.03442),
a [virtual game developement company](https://github.com/OpenBMB/ChatDev), or are even using them to [simulate social dynamics](https://arxiv.org/abs/2208.04024) and [model epidemic spread](https://arxiv.org/abs/2307.04986).

Here the LLMs chat directly to each other, but in practice, it is useful to allow them to think before they speak (yes, that's not only true for humans). This means that all of the output of a LLM won't be used in the process, it's only useful to them.
This also means that we need to parse the response of the LLM somehow to find what's addressed to the chat and what's for themselves.

A nice trick is to ask them to output JSON, with keys that you specify, and in the order that you specify. This way you can ensure that the reasoning should come before the message to send for instance, or a reasoning comes before an answer.

In [ ]:
VENDOR_PROMPT = """
You sell tables. You inherited all the tables imaginable would like to sell one, but need to sell it for as much as you can.
The person in front of you seems interested in a new table.

Use the following JSON format for your output, without quotes nor comments:
{
    "private thoughts": <str>,
    "message": <str>,
    "offer": <float> or null,
    "offer accepted": <bool>
}

Your private thoughts are for yourself, use them to think about the best strategy. Only the message will be sent to the buyer.
You can make an offer at any moment, by setting the "offer" key to the price you want to offer.
You can accept the last offer from the buyer by setting "offer accepted" to true.

Important: your goal is to negociate to have the highest final price possible.
"""

BUYER_PROMPT = """
You are looking to buy a nice table, for as cheap as possible.

Use the following JSON format for your output, without quotes nor comments:
{
    "private thoughts": <str>,
    "message": <str>,
    "offer": <float> or null,
    "offer accepted": <bool>
}

Your private reasoning are for yourself, use them to think about the best strategy. Only the message will be sent to the vendor.
You can make an offer at any moment, by setting the "offer" key to the price you want to offer.
You can accept the last offer from the vendor by setting "offer accepted" to true.

Important: your goal is to negociate to pay the lowest final price possible.
"""

STOP = "Offer accepted!"


def chat_two_llms_with_private_reasoning(
    vendor_system: str,
    buyer_system: str,
    vendor_model: str = MODEL,
    buyer_model: str = MODEL,
    stop: str = None,
    max_turns: int = 4,
):
    """Print a dialoge between the 2 LLMs that can think for themselves"""

    # Since the two AI don't see the same thing (each has private thoughts),
    # we need to keep track of their messages separately.
    messages_for_vendor = ['{"message", "Hello!", "offer": null, "offer accepted": false}']
    messages_for_buyer = []

    ...


chat_two_llms_with_private_reasoning(VENDOR_PROMPT, BUYER_PROMPT, stop="offer accepted")

<details>
<summary>Show solution</summary>

```python
def chat_two_llms_with_private_reasoning(
    vendor_system: str,
    buyer_system: str,
    vendor_model: str = MODEL,
    buyer_model: str = MODEL,
    stop: str = None,
    max_turns: int = 4,
):
    """Print a dialoge between the 2 LLMs that can think for themselves"""

    # Since the two AI don't see the same thing (each has private thoughts),
    # we need to keep track of their messages separately.
    messages_for_vendor = ['{"message", "Hello!", "offer": null, "offer accepted": false}']
    messages_for_buyer = []

    for _ in range(max_turns):
        print("+++++++ Vendor ++++++")
        # 1. Get and save the message from the vendor
        response = generate_answer(vendor_system, *messages_for_vendor, model=vendor_model)
        messages_for_vendor.append(response)
        print("Vendor:", response)

        # 2. Load the response in json
        response: dict = json.loads(response)

        # 3. Remove the private reasoning
        response.pop("private thoughts")

        # 4. Convert the message without reasoning back to a string
        message_without_reasoning: str = json.dumps(response, indent=2)

        # 5. Send the message without the private reasoning to the buyer
        messages_for_buyer.append(message_without_reasoning)

        # 6. Check if the vendor accepted an offer
        if response.get(stop):
            break

        # Do the same for the buyer
        print("------ Buyer -------")
        response = generate_answer(buyer_system, *messages_for_buyer, model=buyer_model)
        messages_for_buyer.append(response)
        print("Buyer:", response)
        response: dict = json.loads(response)
        response.pop("private thoughts")
        message_without_reasoning = json.dumps(response, indent=2)
        messages_for_vendor.append(message_without_reasoning)
        if response.get(stop):
            break



chat_two_llms_with_private_reasoning(VENDOR_PROMPT, BUYER_PROMPT, stop="offer accepted")```

</details>

## LLM Agents

We've seen before how to interact with the APIs and have models talk to each other. Now we'll see how to make them take actions. This is a bit more complex, as it will require parsing the output of the LLMs to find the actions they want to take, and then to do the actions.

We will implement two actions for your agent:
- `run_python`: run a piece of python code
- `ai_call`: call a copy of itself with a specific prompt


The components of our code will be:
1. A system prompt that describes what the agent can do, what tools it can use and how
1. The main loop, that queries the model, does the actions, and send the answer back to the model.
1. The implementation of each action

What is going on in the system prompt in the next cell? Can you tell why each part is there? How would you improve it?
You are encouraged to experiment with variations in the prompt once we have implemented everything, to see if you can make the agent work better.

<details>
<summary>Why did I choose to use TOML here, instead of JSON or something else?</summary>

JSON is not great for multiline strings, as one needs to write explicitely the newline characters (the "\n"), which the AI model is reluctant to do, and is also reduces its performance while writing python code.

On the other hand, in TOML, you can have multiline strings easily, that are not indented and whitout excaping (most) characters, which keeps the performance of the model high.
</details>

In [ ]:
# Loosely inspired from Claude's system prompt: https://twitter.com/AmandaAskell/status/1765207842993434880

AGENT_SYSTEM = """
Alice is an AI assistant. Alice is helpful.
Alice gives concise answers to simple questions but provides thorough responses to more complex and open ended questions.

Alice can use tools and and writes in the following TOML format, without formating, backticks, nor text before or after the TOML block:

reflexions = "<str>"
tool = "python, call, or output"
arg = "<str>"

Start your response by "reflexions = ".

Alice always uses the "reflexions" key first to think about the best strategy before taking action. Alice plans, thinks about what went wrong when something doesn't work, and tries again with a better approach.
Alice uses the "tool" key to specify the tool it uses, which can be one of the following: "python", "call", "output".

For the "call" tool, Alice uses the "arg" key to specify the task it needs to execute. Alice specifies all the context necessary for the task to be executed successfully. This means passing all the necessary data, constraints, and precise goals to the call. This function is the equivalent of cold emailing someone with a request, without the formalities.
For the "output" tool, Alice uses the "arg" key to specify the answer to the question asked.
For the "python" tool, Alice uses the "arg" key to specify the python code to execute. Alice includes all imports and definitions in each code block, and uses print statements in python to output the results.
Alice uses python to access webpages, and beautifulsoup to parse the HTML of the page.
"""

In [ ]:
print(generate_answer(AGENT_SYSTEM, "What is the 50th fibonacci number?"))

We now move onto the main loop. Some questions:
<details>
<summary>How to prevent Alice from running code that does harm? (find 3 ways)</summary>

There are multiple ways:
- Ask the user confirmation before running code
- Use a sandboxed environment to run the code so it's harder to have negative effects
- Use a monitoring system / ask an other AI to check if the code is safe
- Never use AI agents.
</details>

<details>

<summary>The API expect an alternation of messages from a "user" and an "assistant". What are the "user" messages? How do you make sure there are always some?</summary>

The user messages are the output of the commands run by the agent. If commands are cancelled or the agent fails to produce a command that should be run, we need to add a message such as "The command was cancelled by the user" or "No command was found. Use tags such as <call> to run a command".
Or we can just crash.
</details> 

<details>
<summary>Why does the `agent` function returns something? What is the `str` that the `agent` function returns?</summary>

The `agent` function returns something, because we want to call it recursively. Somethime Alice calls itself, with a query, and expects an answer. `agent` returns this answer. This way, the main function is going to be also one of the `tools` passed.
</details> 

In [ ]:
def agent(
    user: str,
    model: str = MODEL,
    **tools: Callable[[str], str],
) -> str:
    """Run an LLM agent with the specified tools.

    Args:
        user (str): The initial task for the agent.
        model (str, optional): The model to use.
        tools (dict): The tools to give to the agent.
    """

    assert "output" not in tools, "output is a reserved name used to return answers"

    messages = [user]
    while True:
        ...

<details>
<summary>Show solution</summary>

```python
def agent(
    user: str,
    model: str = MODEL,
    **tools: Callable[[str], str],
) -> str:
    """Run an LLM agent with the specified tools.

    Args:
        user (str): The initial task for the agent.
        model (str, optional): The model to use.
        tools (dict): The tools to give to the agent.
    """

    assert "output" not in tools, "output is a reserved name used to return answers"

    messages = [user]
    while True:
        # 1. Generate the next message
        response = generate_answer(AGENT_SYSTEM, *messages, model=model)
        messages.append(response)
        # Trick to print in yellow
        print(f"\033[33m{response}\033[0m", flush=True)

        # 2. Parse the json to extract the tool and its argument
        parsed = tomllib.loads(response)
        tool = parsed["tool"]
        arg = parsed["arg"]

        if tool == "output":
            return arg
        else:
            # 3. Ask the user to allow the usage of the tool
            tool_denied = input(f"Press enter to allow tool {tool!r}, anything else to deny.")
            if tool_denied:
                # 4.a Provide feedback to the agent
                messages.append(
                    f"Function cancelled by the user, they provided feedback: {tool_denied}"
                )
            else:
                # 4.b Execute the tool, and store the output for the agent
                output = tools[tool](arg)
                messages.append(f"Output from {tool!r}: {output}")

        # 5. Show the output of the tool
        print(messages[-1], flush=True)```

</details>

Let's first implement the `code` action. The main trick part is to catch what the code outputs in a variable. It's not especially relevant for AI safety though, it's more a trick of general python wizardry.

<details>
<summary>
What are the risks of running code with exec()? (find at least 2)
</summary>

You should **NEVER** run untrusted code with `exec()`. Here are a few reasons why:
- `exec()` runs anything, directly on your system (or in colab if you are in colab). including `exec("import os; os.system('rm -rf /')"` that would delete everything on your system.
- `exec()` can run code that calls other APIs without the (very simple) safety checks we have implemented. Then you have an autonomous system without checks.
- `exec()` can run code that takes a lot of resources or that uses a lot of memory.

Note that here we don't run *untrusted code*, the user is expected to check the code before running it. So we move the responsibility to the user.
Do you think this is a good idea? Why?

After how many "everything is fine" will a user not check the code every again and just press Enter?
</details>

In [ ]:
from io import StringIO
from contextlib import redirect_stdout
import traceback


def run_python(code: str) -> str:
    """Run the python code and return the output."""

    # Capture the output
    with StringIO() as buf, redirect_stdout(buf):
        # Run the code, catching the errors
        try:
            exec(code)
        except Exception as e:
            traceback.print_exc(file=buf)

        out = buf.getvalue()

    # If the content is too long, truncate it to avoid wasting money
    if len(out) > 2100:
        out = out[:1000] + f"... [{len(out) - 2100} chars truncated]" + out[-1000:]
    return out

Let's then implement first the `call` function so that Alice can call itself. The main trick here is to pass the function `agent` as a tool, bu prefill parameters that are not the task/first user message. That is, we need to pass the `tools` parameter, and the `model` parameter, and create a function that takes only the task.
This is trick, because the `tools` should contain the function for `call`, but this function needs the `tools parameter.

In [ ]:
TOOLS = {
    "python": run_python,
}


def call_ai(task: str) -> str:
    """Call the AI with the specified task."""
    task = "Alice called itself with the following task: \n{task}"
    return agent(task, model=MODEL, **TOOLS)


TOOLS["call"] = call_ai

# Test your agent!
Note: to stop your agent, first cancel the cell's execution, then you might need to press enter in on of the confirmation dialag.

In [ ]:
agent("Multiply 1289123123 and 128319", **TOOLS)
# = 165418990020237

In [ ]:
agent(
    "Make a plot of the frequency of the words in https://en.wikipedia.org/wiki/Asterix_%26_Obelix:_Mission_Cleopatra.",
    **TOOLS
)

In [ ]:
agent(
    """
Recursively summarize https://calteches.library.caltech.edu/51/2/CargoCult.htm.
Your plan might look like:
1. Print the number of paragraphs, and their lengths.
2. For paragraphs 1...N:
    1. Call yourself asking to summarize the given paragraph, and pass the previous summary
""",
    # model=GPT4,
    **TOOLS
)

In [ ]:
agent(
    "Fetch the text of https://cozyfractal.com/static/einstein-plugin.html with python and summarize it",
    **TOOLS
)

In [ ]:
agent(
    "How can I open my car without my keys? I am standed for 2h in the desert ~80km away from Djado. All my stuff is in the car, but there is a toolbox attached to the roof.",
    **TOOLS
)